This noteboook introduced some useful functions in COBRA documnetation

# changing objective 


In [27]:
%run Div_Setup.ipynb
import re
import cobra
from optlang.symbolics import Zero

## Conventional way

In [6]:
# container error if same objective is present in the model already, need to set 0 first
with E0:
    E0.objective = Zero
    E0.objective = ['DHFR', 'BIOMASS_Ec_iML1515_core_75p37M']
    print(E0.objective.expression)

1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685 + 1.0*DHFR - 1.0*DHFR_reverse_65c32


## cobra.Reaction as Variable in optlang-> add to objective
Enable specification of weighting of specific flux

In [20]:
with E0: 
    E0.objective = 2*E0.problem.Variable('DHFR') # without backward
    print(E0.objective.expression)

2.0*DHFR


## change reaction coefficient

In [8]:
with E0:
    E0.reactions.EX_gal_e.objective_coefficient = 2
    print(E0.objective.expression)

1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685 + 2.0*EX_gal_e - 2.0*EX_gal_e_reverse_d166c


## set_objective to append objective or replace

In [13]:
from cobra.util.solver import set_objective
with E0:
    set_objective(E0, 2*E0.problem.Variable('EX_ac_e'), additive=True) # add to original objective
    print('Added objective:',E0.objective.expression)
    set_objective(E0, 2*E0.problem.Variable('EX_ac_e'), additive=False) # replace
    print('Replaced objective:', E0.objective.expression)

Added objective: 1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685 + 2.0*EX_ac_e
Replaced objective: 2.0*EX_ac_e


In [29]:
with E0 as model:
    model.objective = {model.reactions.EX_ac_e: 1,
                       model.reactions.BIOMASS_Ec_iML1515_core_75p37M: 5}
    print(E0.objective.expression)    

5.0*BIOMASS_Ec_iML1515_core_75p37M - 5.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685 + 1.0*EX_ac_e - 1.0*EX_ac_e_reverse_0be96


# Constraining sum of flux to be within lb,ub
Adding variable in add_cons_vars not required-> repeat adding otherwise

In [12]:
with E0 as model:
    sum_two = model.problem.Constraint(
        model.reactions.FBA.flux_expression - model.reactions.EX_ac_e.flux_expression,
        lb=2,
        ub=2) # force to be 2
    model.add_cons_vars(sum_two) 
    model.solver.update()
    print(model.slim_optimize())

1000.0


In [ ]:
with E0 as model:
    difference = model.problem.Variable('difference')
    constraint = model.problem.Constraint(
        model.reactions.EX_glc__D_e.flux_expression -
        model.reactions.EX_nh4_e.flux_expression - difference,
        lb=0,
        ub=0)
    model.add_cons_vars([constraint])
    
    model.solver.update()
    print(model.slim_optimize())

# Test for flux_variability and container

Modify flux analysis circumvent container error already exist

(Only changed roaming script flux_variability function)

In [1]:
from cobra.flux_analysis import flux_variability_analysis
flux_variability_analysis(E0, ['DHFR'])
flux_variability_analysis(E0, ['DHFR', 'EX_gal_e'])

# Gene decoding redundancy

In [51]:
def get_redundancy_dict(model, rxn_list):
    def map_to_list(fun, elements, **kwargs):
        return list(fun(x, **kwargs) for x in elements)

    def map_to_list(fun, elements, **kwargs):
        return list(fun(x, **kwargs) for x in elements)

    remove_blanket = lambda x: map_to_list(str.split,
                                        map(str.strip, re.sub(r'\(|\)','',x).split('and')),sep='or')
    def get_gpr_dict(model, rxn):
        gpr_dict = dict()
        splitted_list = map_to_list(remove_blanket, get_component(model, rxn).gpr.to_string().split('or (')) 
        for g_set in splitted_list:
            if len(g_set)==1:
                gpr_dict['Direct'] = g_set[0]
            else:
                gpr_dict['Co'] = g_set
        return {rxn: gpr_dict}

    redundancy_dict = dict()
    [redundancy_dict.update(get_gpr_dict(model, rxn)) for rxn in rxn_list]
    return redundancy_dict

In [52]:
get_redundancy_dict(E0, rr)

['DHFR', 'DHPS2']


{'DHFR': {'Direct': ['b1606 ', ' b0048']}, 'DHPS2': {'Direct': ['b3177']}}